<a href="https://colab.research.google.com/github/poojalanghnoda/hackerrank_leaderboard/blob/master/hackerrank__contest_leaderboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import getpass
import sys


import requests

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

def print_get_contest(email, password):

    contestsr = requests.get(
        "https://www.hackerrank.com/rest/administration/contests",
        params={
            "offset": 0,
            "limit": 40   # 40 contest data downloaded. If there are more constents than change it here.
        }, auth=(email, password), headers=headers)
    
    if contestsr.status_code != 200:
        print("Error getting contests. Exiting...")
        sys.exit(-1)

    contests_dict = contestsr.json()

    if not contests_dict['status']:
        print("No success getting contests. Exiting...")
        sys.exit(-1)

    contests_list = contests_dict['models']

    for i in range(len(contests_list)):
        print("%d) %s (%s)" %
              (i+1, contests_list[i]['name'], contests_list[i]['slug']))

    c_no = int(input("Enter contest no :"))

    if c_no-1 >= len(contests_list):
        print("Invalid contest no. Exiting...")
        sys.exit(-1)

    return contests_list[c_no-1]['slug'], contests_list[c_no-1]['id']


def get_leaderboard_data(email, password, slug):
    lbr = requests.get(
        "https://www.hackerrank.com/rest/contests/" + slug + "/leaderboard",
        params={
            "offset":0,
            "limit": 500  #500 students data will be fetch for a selected contests. if more students than update it here.
        }, auth=(email, password), headers=headers)
    
    if lbr.status_code != 200:
        print("Error getting leaderboard. Exiting...")
        sys.exit(-1)
    
    lb_list = lbr.json()['models']
    return lb_list


def get_contest_details(email, password, slug, id):
    time_detailsr = requests.get(
        "https://www.hackerrank.com/rest/administration/contests/" + str(id),
        auth=(email, password), headers=headers)

    if time_detailsr.status_code != 200:
        print("Error getting contest time details. Exiting...")
        sys.exit(-1)

    time_dict_json = time_detailsr.json()

    if not time_dict_json['status']:
        print("Failure while getting contest time details. Exiting...")
        sys.exit(-1)

    time_dict = time_dict_json['model']

    total_time = time_dict['endtime'] - time_dict['starttime']

    score_detailsr = requests.get(
        "https://www.hackerrank.com/rest/administration/contests/"
        + str(id) + "/challenges",
        params={
            "offset": 0,
            "limit": 200
        }, auth=(email, password), headers=headers)

    if score_detailsr.status_code != 200:
        print("Error getting contest score details. Exiting...")
        sys.exit(-1)

    score_dict_json = score_detailsr.json()

    if not score_dict_json['status']:
        print("Failure while getting contest score details. Exiting...")
        sys.exit(-1)

    challenges_list = score_dict_json['models']

    scores = []

    for c in challenges_list:
        scores.append(c['weight'])

    return total_time, scores


def get_leaderboard_file(email, password):
    con_slug, con_id = print_get_contest(email, password)
    ttime, scores = get_contest_details(email, password, con_slug, con_id)
    leaderboard_data = get_leaderboard_data(email, password, con_slug)
    total_score = sum(scores)
    with open('leaderboard-' + con_slug + ".csv", 'w') as lbf:
        fieldnames = ['rank', 'username', 'score', 'normalized_score',
                      'time_in_sec', 'normailzed_time']
        writer = csv.DictWriter(lbf, fieldnames=fieldnames)
        writer.writeheader()

        for lb_entry in leaderboard_data:

            # 100 points
            norm_score = lb_entry['score']/total_score*100
            # 1 hour
            norm_time = lb_entry['time_taken']/(ttime*len(scores))*3600

            writer.writerow({
                'rank': lb_entry['rank'],
                'score': lb_entry['score'],
                'normalized_score': norm_score,
                'time_in_sec': int(lb_entry['time_taken']),
                'normailzed_time': norm_time,
                'username': lb_entry['hacker']
            })

    return True


def main():
  email = input("Enter email :")
  password = getpass.getpass("Enter password (hidden) :")
  
  if get_leaderboard_file(email, password):
    print("Done")


if __name__ == '__main__':
    main()


Enter email :introtoprogramming105@gmail.com
Enter password (hidden) :··········
1) Lab3Group2 (lab3group2)
2) Lab3Group3 (lab3group3)
3) Lab3Group4 (lab3group4)
4) Lab3Group5 (lab3group5)
5) Lab3Group1 (lab3group1)
6) Lab4Group2 (lab4group2)
7) Lab4Group3 (lab4group3)
8) Lab4Group4 (lab4group4)
9) Lab4Group5 (lab4group5)
10) Lab4Group1 (lab4group1)
11) Lab5Group2 (lab5group2)
12) Lab6Group2 (lab6group2)
13) lab7group2 (lab7group2-12)
14) Lab8Group2 (lab8group2-10)
15) Lab8group3 (lab8group3)
16) Lab8Group4 (lab8group4)
17) Lab8Group5 (lab8group5)
18) Lab8Group1 (lab8group1)
19) Lab9Group2 (lab9group2)
20) Lab9Group3 (lab9group3)
21) Lab10Group2 (lab10group2-link)
22) Lab10Group4 (lab10group4)
23) Lab10Group5 (lab10group5)
24) Lab10Group1 (lab10group1-56)
25) Lab10Group3 (lab10group3)
26) endsem (endsem)
27) sampletesting (sampletesting-1)
28) sampleterstend (sampleterstend)
29) End Sem Exam Odd (end-sem-exam-odd11111)
30) End Sem Exam Even (end-sem-exam-even121212)
31) End Sem Exam Ev